# TRABAJO PRÁCTICO 4 - KNN Y ANÁLISIS DEL DISCRIMINANTE

In [205]:
#librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seaborn import pairplot 
from sklearn import model_selection as ms

## Exploración de datos

In [206]:
#Se cargan los datos del csv
df = pd.read_csv('MulticlassDiabetesDataset.csv')

p_test = 0.2
p_train = 0.8

#Cantidad de muestras por clase
# print(df.shape[0])

#Gráfico con pairplot
# pairplot(df)

#Se definen los conjuntos de entrenamiento y testeo 
[train,test] = ms.train_test_split(df, train_size = p_train ,test_size = p_test)

y_train=train['Class'].sort_index()
x_train = train.drop(columns = ['Class']).sort_index()

y_test = test['Class'].sort_index()
x_test = test.drop(columns = ['Class']).sort_index()

n_cat = len(np.unique(y_train))

print(x_train.shape)
print(y_train.shape)
print(n_cat)


(211, 11)
(211,)
3


## Análisis del discriminante

In [ ]:
class LDA_QDA:
    def __init__ (self, LDA = False, n_cat = 0):
        self.LDA = LDA #Se define si se quiere el modelo LDA o QDA
        self.n_cat = n_cat #Se guarda la cantidad de categorías en el problema
        self.D = None #Clasificaciones
        self.c = None #Probabilidades
        self.sigma = None #Matriz de covarianza
        self.sigma_lda = None #Matriz de covarianza para LDA

    #Testeo de entrenamiento
    def fit (self, X, y):
        
        #Se separan las muestras que corresponden a cada categoría en D (D_k)
        self.D = [X[y==i] for i in range(self.n_cat)]

        n_samples = X.shape[0] #Se guarda la cantidad de muestras
        self.c = []
        self.mu = []
        self.sigma = []
        
        if self.LDA == True:
            self.sigma_lda = []

        for i in range(self.n_cat):
            #Se calculan las constantes para cada conjunto
            self.c.append( self.D[i].shape[0]/n_samples )

            #Se calcula la media de cada categoría en cada feature
            self.mu.append( (1/self.D[i].shape[0]) * np.sum(self.D[i],axis = 0) )

            #Se calcula sigma para cada categoría
            self.sigma.append( (1/(self.D[i].shape[0]-1)) * (self.D[i]-self.mu[i]).T @ (self.D[i]-self.mu[i]) )

            #Se calcula el sigma que se utiliza en el método LDA para cada categoría
            if self.LDA == True:
                self.sigma_lda.append( (1/(n_samples-self.n_cat)) * (self.D[i].shape[0] -1 ) * self.sigma[i] )
        
        if self.LDA == True:
            self.sigma_lda = np.sum(self.sigma_lda,axis = 0) #Se calcula el sigma resultante para utilizar en LDA tras el calculo por categorías
            
        # print(len(self.sigma))
        # print(self.sigma_lda[0].shape)
        # print(self.D[0].shape)
        # print(len(self.mu))
        # print(self.D[0].shape)
        # print(self.D[1].shape)
        # print(self.D[2].shape)
        
        
    # #Testeo soft
    def predict_prob (self, X):
        delta = self.predict_discriminant(X)
        exp_delta = np.exp(delta)
        pred_soft = exp_delta / np.sum(exp_delta, axis=1, keepdims=True) #Se calcula la predicción soft, cuidando de hacer la suma en cada fila de forma correcta
        return pred_soft
        

    # #Testeo hard
    def predict (self, X):
        delta = self.predict_discriminant(X)
        pred_hard = np.argmax(delta,axis=1)
        return pred_hard


    #Alternativa práctica para el testeo soft
    def predict_discriminant(self, X):
        
        if hasattr(X, "values"):
            x = X.values
        else:
            x = np.array(X)

        if x.ndim == 1:
            x = x.reshape(-1, 1)
            
            
        delta = np.zeros((x.shape[0], self.n_cat)) #Se crea un array de los deltas de n_muestras x n_categorias
        if self.LDA == True:
            
            if np.ndim(self.sigma_lda)> 0:        #Para múltiples dimensiones
                inv_sigma_lda = np.linalg.inv(self.sigma_lda) #Se calcula la inversa de la matriz sigma para evitar repetir cálculos
                for j in range(x.shape[0]): #Se calcula el discriminante en cada x, en las n_cat
                    for i in range(self.n_cat):
                        delta[j,i] = np.log(self.c[i]) + x[j,:].T @ inv_sigma_lda @ self.mu[i] - 0.5 * self.mu[i].T @ inv_sigma_lda @ self.mu[i]
            
            else:  #Para 1 dimensión
                inv_sigma_lda = 1.0/self.sigma_lda #Se calcula la inversa de la matriz sigma para evitar repetir cálculos
                for j in range(x.shape[0]): #Se calcula el discriminante en cada x, en las n_cat
                    for i in range(self.n_cat):
                        delta[j,i] = np.log(self.c[i]) + x[j,:].T @ inv_sigma_lda @ self.mu[i] - 0.5 * self.mu[i].T @ inv_sigma_lda @ self.mu[i]

        else: 
            if np.ndim(self.sigma_lda) > 0:
                for j in range(x.shape[0]): #Se calcula el discriminante para cada muestra en las n features, usando QDA
                    for i in range(self.n_cat):
                        diff = x[j,:]-self.mu[i] #Se calcula la diferencia para cada muestra en las n categorías para evitar repetir cálculos
                        delta[j,i] = np.log(self.c[i]) - (0.5) * diff.T @ np.linalg.inv(self.sigma[i]) @ (diff) - 0.5 * np.log(np.linalg.det(self.sigma[i]))
                
        return delta

In [208]:
#Se selecciona el feature 'HbA1c' para entrenar el modelo
selected_feature_train = np.array(x_train['HbA1c'])

#Entrenamiento LDA
model_lda = LDA_QDA(LDA = True, n_cat = 3)

model_lda.fit(selected_feature_train,y_train)

#Entrenamiento QDA
model_qda = LDA_QDA(n_cat = 3)

model_qda.fit(selected_feature_train,y_train)

#Se grafica la Función Discriminante

x_min, x_max = selected_feature_train.min() - 1, selected_feature_train.max() + 1
y_min, y_max = selected_feature_train.min() - 1, selected_feature_train.max() + 1
    
    # Crear malla
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    # Evaluar discriminante en cada punto de la grilla
grid_points = np.c_[xx.ravel(), yy.ravel()]
delta = model_lda.predict_discriminant(grid_points)
    
# Clasificación dura en la grilla
Z = np.argmax(delta, axis=1)
Z = Z.reshape(xx.shape)

# Dibujar frontera con contourf
plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.tab10)

# También graficar líneas de nivel de discriminantes
for i in range(model_lda.n_cat):
    plt.contour(xx, yy, delta[:, i].reshape(xx.shape), 
                levels=5, alpha=0.5, cmap="coolwarm")

# Puntos originales
scatter = plt.scatter(selected_feature_train[:, 0], selected_feature_train[:, 1], c=y_train, cmap=plt.cm.tab10, edgecolor="k")
plt.legend(*scatter.legend_elements(), title="Clases")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.title("Fronteras de decisión y funciones discriminantes")
plt.show()


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)